In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from math import cos as cos
from math import sin as sin
from torch.utils.data import TensorDataset
import warnings
warnings.filterwarnings('ignore')

In [ ]:
grid_size = 7
image_size = (64, 64)
num_BB = 2
num_classes = 20
depth = 5 * num_BB + num_classes
num_epoches = 2
Batch_size = 8

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
class YOLOv1(nn.Module):
    def __init__(self):
      super().__init__()
      layers = [
          nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3),
          nn.LeakyReLU(negative_slope=0.1),
          nn.MaxPool2d(kernel_size=2, stride=2),

          nn.Conv2d(64, 192, kernel_size=3, padding=1),
          nn.LeakyReLU(negative_slope=0.1),
          nn.MaxPool2d(kernel_size=2, stride=2),

          nn.Conv2d(192, 128, kernel_size=1),
          nn.LeakyReLU(negative_slope=0.1),
          nn.Conv2d(128, 256, kernel_size=3, padding=1),
          nn.LeakyReLU(negative_slope=0.1),
          nn.Conv2d(256, 256, kernel_size=1),
          nn.LeakyReLU(negative_slope=0.1),
          nn.Conv2d(256, 512, kernel_size=3, padding=1),
          nn.LeakyReLU(negative_slope=0.1),
          nn.MaxPool2d(kernel_size=2, stride=2),

          nn.Conv2d(512, 256, kernel_size=1),
          nn.Conv2d(256, 512, kernel_size=3, padding=1),
          nn.LeakyReLU(negative_slope=0.1),
          nn.Conv2d(512, 256, kernel_size=1),
          nn.Conv2d(256, 512, kernel_size=3, padding=1),
          nn.LeakyReLU(negative_slope=0.1),
          nn.Conv2d(512, 256, kernel_size=1),
          nn.Conv2d(256, 512, kernel_size=3, padding=1),
          nn.LeakyReLU(negative_slope=0.1),
          nn.Conv2d(512, 256, kernel_size=1),
          nn.Conv2d(256, 512, kernel_size=3, padding=1),
          nn.LeakyReLU(negative_slope=0.1),
          nn.Conv2d(512, 512, kernel_size=1),
          nn.Conv2d(512, 1024, kernel_size=3, padding=1),
          nn.LeakyReLU(negative_slope=0.1),
          nn.MaxPool2d(kernel_size=2, stride=2),

          nn.Conv2d(1024, 512, kernel_size=1),
          nn.Conv2d(512, 1024, kernel_size=3, padding=1),
          nn.LeakyReLU(negative_slope=0.1),
          nn.Conv2d(1024, 512, kernel_size=1),
          nn.Conv2d(512, 1024, kernel_size=3, padding=1),
          nn.LeakyReLU(negative_slope=0.1),
          nn.Conv2d(1024, 1024, kernel_size=3, padding=1),
          nn.LeakyReLU(negative_slope=0.1),
          nn.Conv2d(1024, 1024, kernel_size=3, stride=2, padding=1),
          nn.LeakyReLU(negative_slope=0.1),

          nn.Conv2d(1024, 1024, kernel_size=3, padding=1),
          nn.LeakyReLU(negative_slope=0.1),
          nn.Conv2d(1024, 1024, kernel_size=3, padding=1),
          nn.LeakyReLU(negative_slope=0.1),

          nn.Flatten(),
          nn.Linear(7 * 7 * 1024, 4096),
          nn.Dropout(),

          nn.LeakyReLU(negative_slope=0.1),
          nn.Linear(4096, 7 * 7 * depth)
      ]

      self.layers = nn.Sequential(*layers)

    def forward(self, x):
      x = self.layers(x)
      return torch.reshape(x,(x.shape[0], 7, 7, depth))

In [ ]:
class YOLOv1Loss(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, Y_pred, Y):
        batch_size = Y.size(0)
        loss = 0

        for i in range(batch_size):
            for y in range(grid_size):
                for x in range(grid_size):

                  loss += 0.5 * Y[i, x, y, 0] * ((Y_pred[i, x, y, 1] - Y[i, x, y, 1])**2 + (Y_pred[i, x, y, 2] - Y[i, x, y, 2])**2)
                  loss += 0.5 * Y[i, x, y, 5] * ((Y_pred[i, x, y, 6] - Y[i, x, y, 6])**2 + (Y_pred[i, x, y, 7] - Y[i, x, y, 7])**2)

                  loss += 0.5 * Y[i, x, y, 0] * ((abs(Y_pred[i, x, y, 3])**0.5 - Y[i, x, y, 3]**0.5)**2 + (abs(Y_pred[i, x, y, 4])**0.5 - Y[i, x, y, 4]**0.5)**2)
                  loss += 0.5 * Y[i, x, y, 5] * ((abs(Y_pred[i, x, y, 8])**0.5 - Y[i, x, y, 8]**0.5)**2 + (abs(Y_pred[i, x, y, 9])**0.5 - Y[i, x, y, 9]**0.5)**2)

                  if(Y[i, x, y, 0]==0):
                    loss += 0.5 * (Y[i, x, y, 0] - Y_pred[i, x, y, 0])**2
                  else:
                    loss += (Y[i, x, y, 0] - Y_pred[i, x, y, 0])**2

                  if(Y[i, x, y, 5]==0):
                    loss += 0.5 * (Y[i, x, y, 5] - Y_pred[i, x, y, 5])**2
                  else:
                    loss += (Y[i, x, y, 5] - Y_pred[i, x, y, 5])**2

                  for j in range(10, 10+num_classes):
                    loss += Y[i, x, y, 0] * ((Y[i, x, y, j]-Y_pred[i, x, y, j])**2)
        return loss

In [ ]:
model = YOLOv1().to(device)

In [ ]:
loss_function = YOLOv1Loss()

In [ ]:
optimizer = torch.optim.SGD(
        model.parameters(),
        lr=1e-4,
        momentum=0.9,
        weight_decay=0.0005
    )

In [ ]:
validation_loss = torch.load('/content/drive/MyDrive/VOC2007/Validation/real_loss.pt').to(device)
validation_loss = list(validation_loss)

In [ ]:
training_loss = torch.load('/content/drive/MyDrive/VOC2007/Train/real_loss.pt').to(device)
training_loss = list(training_loss)

In [ ]:
train_loader = DataLoader(TensorDataset(torch.load("/content/drive/MyDrive/VOC2007/Train/Batch3/train_imgs.pt").to(device), torch.load("/content/drive/MyDrive/VOC2007/Train/Batch3/train_gt.pt").to(device)), batch_size=Batch_size)

In [ ]:
val_loader = DataLoader(TensorDataset(torch.load("/content/drive/MyDrive/VOC2007/Validation/Batch1/val_imgs.pt").to(device), torch.load("/content/drive/MyDrive/VOC2007/Validation/Batch1/val_gt.pt").to(device)), batch_size=Batch_size)

In [ ]:
model.load_state_dict(torch.load('/content/drive/MyDrive/VOC2007/real_model_weights.pt'))

In [ ]:
for epoch in range(1, num_epoches):

  model.train()
  train_loss = 0
  for batch_idx, (X, Y) in enumerate(train_loader):

    optimizer.zero_grad()
    X = torch.permute(X, (0, 3, 1, 2))
    predictions = model.forward(X)
    predictions = torch.clamp(predictions, min=0, max=1)
    Loss = loss_function(predictions, Y)
    Loss.backward()
    optimizer.step()
    train_loss += Loss

  torch.save(model.state_dict(), '/content/drive/MyDrive/VOC2007/real_model_weights.pt')
  print("training loss: epoch: ", epoch, "loss: ", train_loss/1500)
  training_loss.append(train_loss/1500)
  torch.save(torch.Tensor(training_loss).to(device), '/content/drive/MyDrive/VOC2007/Train/real_loss.pt')

  model.eval()
  val_loss = 0
  with torch.inference_mode():
    for batch_idx, (X, Y) in enumerate(val_loader):
      X = torch.permute(X, (0, 3, 1, 2))
      predictions = model(X)
      val_loss += loss_function(predictions, Y)

  print("validation loss: epoch: ", epoch, "loss: ", val_loss/100)
  validation_loss.append(val_loss/100)
  torch.save(torch.Tensor(validation_loss).to(device), '/content/drive/MyDrive/VOC2007/Validation/real_loss.pt')

In [ ]:
len(training_loss)

In [ ]:
len(validation_loss)

In [ ]:
training_loss

In [ ]:
validation_loss

In [ ]:
torch.save(torch.Tensor(validation_loss).to(device), '/content/drive/MyDrive/VOC2007/Validation/real_loss.pt')